# Using VQE to determine the ground state of an open 3-qubit chain

## Introduction

This notebook is a complementation to on of my PhD projects, where I investigate what happens to a chain of three qubits after the end-tip qubits is submit to periodic measurements (= "punches").

Here I implement a very basic instance of the Variational Quantum Eigensolver (VQE) in order to find the ground state of this Hamiltonian for the sake of comparison in the actual project.

## VQE

First, let us create a function that uses the `Estimator` primitive to compute the expectation value of our Hamiltonian
\begin{equation}
    E(\vec \theta) = \langle \psi(\vec \theta) | H | \psi(\vec \theta) \rangle.
\end{equation}
where, as usual, $\vec \theta$ is a set of parameters which we would like to investigate in order to minimize $E(\vec \theta)$.

In [1]:
def vqe_energy(params, ansatz, hamiltonian, estimator):

    ## Primitive unitary blocks
    pubs = (ansatz, hamiltonian, params)

    ## Running the job
    job = estimator.run([pubs])
    result = job.result()[0]

    ## Returning expectation value
    return result.data.evs

Now, considering that we don't have any prior clue about the set of _optimal parameters_ $\vec \theta^*$ which will minimize this Hamiltonian and solve our problem, we create a 6-parameter ansatz corresponding to the six rotation angles describing the qubits in their respective Bloch spheres (two for each qubit). We also define our Hamiltonian and use `scipy.minimize` to implement the optimization routine.

In [3]:
import numpy as np
from scipy.optimize import minimize
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.primitives import StatevectorEstimator
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumCircuit, ParameterVector

## Creating parameter vector
theta = ParameterVector("θ", 6)

## Defining our ansatz and applying parameterized gates
ansatz = QuantumCircuit(3)

## Applyting parameterized gates

# qubit 0
ansatz.rz(theta[0], 0)
ansatz.rx(theta[1], 0)

# qubit 1
ansatz.rz(theta[2], 1)
ansatz.rx(theta[3], 1)

# qubit 2
ansatz.rz(theta[4], 2)
ansatz.rx(theta[5], 2)

## Hamiltonian
J_1 = J_2 = 2.0
H = SparsePauliOp(data = ["XXI", "IZZ"], coeffs = [-J_1/2, -J_2/2])

## Calling Estimator
estimator = StatevectorEstimator()

## Optimization routine

# Initial guess
x0 = np.ones(6) # Initial set of parameters

# SciPy local BFGS (gradient-based) minimizer
result = minimize(vqe_energy, x0, args = (ansatz, H, estimator), method = "BFGS")

## Storing optimal parameters
opt_params = result.x
opt_params[opt_params < 1e-1] = 0 # Setting very small parameters to zero

## Assigning parameters to the ansatz and printing the resulting statevector
ansatz = ansatz.assign_parameters({theta: opt_params})
Statevector(ansatz).draw("latex")

<IPython.core.display.Latex object>